# Régression logistique

## 1 Régularisation de de Tikhonov

### Question 1.1

$$
f_1: (w_0, w) \mapsto \frac{1}{n}\sum_{i=1}^nlog(1 + e^{-y_i(x_i^Tw + w_0)}) + \frac{\rho}{2}\|w\|_2^2
$$

#### Gradient de $f_1$

$$
\frac{\partial f_1}{\partial w_0}(w_0, w) = \frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}}
$$

$$
\frac{\partial f_1}{\partial w}(w_0, w) = \frac{1}{n}\sum_{i=1}^n\frac{-y_ix_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}} + \rho w
$$

$$
\nabla f_1(w_0, w) = \begin{pmatrix}\frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}} \\  \frac{1}{n}\sum_{i=1}^n\frac{-y_ix_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}} + \rho w\end{pmatrix}
$$

#### Matrice hessienne de $f_1$

#### Convexité de $f_1$